# Monitoring

Monitoring is the process of observing application behavior to ensure reliable operation.

Logging is the process of storing information for monitoring, debugging or analysis purposes.

## Logging

Logging is done by writing log statements when the program is being executed. Logs are not intended to influence the behavior of application, but rather to help understand what is happening for the observers.

```csharp
_logger.LogInformation("We are hereeeee");
```

### Log severity

Logs are usually divided into different severity levels:

1. Trace
1. Debug
1. Information
1. Warning
1. Error

Log levels are intended to help engineers better filter information that is being produced. Each subsequent level goes up in severity. More severe levels can be interpreted as being more important.

For example if `Trace` levels logs are very verbose, where they can even be made to print information about each statements that is being executed. Such verbosity can be too much even for local debugging purposes, so the engineer can choose not to print or display `Trace` level logs.

Similarly we could say that `Error` logs are so important and mean such extraordinary situations, that some engineer should be automatically notified when this happens. This report would be an example of alert.

### Logging in ASP.NET

ASP.NET generic host comes prepared with a console logger provider. To access this logger, you simply need to inject it into your class:

```csharp
public class MyClass
{
    private readonly ILogger _logger;

    public MyClass(ILogger<MyClass> logger)
    {
        _logger = logger;
    }
}
```

Note that `ILogger<>` must be injected in it's generic form, there the generic type matches the type into which you are injecting it.

`ILogger` has methods for all severities and overloads for most common scenarios:

```csharp
_logger.LogDebug("Debug");
_logger.LogInformation("Information");
_logger.LogWarning("Warning");
```

Severity can also be passed as an argument:

```csharp
_logger.Log(LogLevel.Information, "Information");
```

Overload to pass an exception for an error log:

```csharp
_logger.LogError(new Exception(), "Exception happened");
```

## Monitoring tools

One of the things that enable monitoring are tools that allow to collect, visualize and act upon metrics provided by the program.

### Metrics

- Counter: value that only goes up.
- Gauge: momentary value.
- Histogram: event type value that might be statistically significant.

## Monitoring example with Grafana

This example will show sample setup for using Grafana for logs, dashboards and alerting together with ASP.NET.

```mermaid
graph TD
    A[ASP.NET Application] -->|Telemetry Data| B[OpenTelemetry .NET SDK]
    B -->|Exports Metrics| C[OpenTelemetry Collector]
    C -->|Scraped Metrics| D[Prometheus]
    D -->|Serves Data| E[Grafana]
    E -->|Visualizes Data| F[Dashboards]

    subgraph Observability Pipeline
        B
        C
        D
        E
    end
```

### Providing metrics

To be able to display something in external tools, first we will need to start outputting some information - metrics - that later could be used for display.

#### Providing metrics with `OpenTelemetry`

`OpenTelemetry` is a set packages that can be used to export metrics for monitoring tools in various format.

To start using it, add the following packages:

```bash
dotnet add package OpenTelemetry.Exporter.Prometheus.AspNetCore --prerelease
dotnet add package OpenTelemetry.Extensions.Hosting
dotnet add package OpenTelemetry.Instrumentation.AspNetCore
```

In the API project register the OpenTelemetry services to the DI and add the middlewares:

Dependency injection:

```csharp
builder
    .Services.AddOpenTelemetry()
    .WithMetrics(metrics =>
    {
        metrics.AddPrometheusExporter().AddAspNetCoreInstrumentation();
        metrics.SetResourceBuilder(ResourceBuilder.CreateDefault().AddService("dotnet-app"));
    });
```

Middlewares:

```csharp
// This by default will start exposing metrics at /metrics endpoint.
app.MapPrometheusScrapingEndpoint();
```


### Reading metrics with Prometheus

Create `prometheus.yml` as follows:

```yaml
global:
  scrape_interval: 5s

scrape_configs:
  - job_name: "dotnet-app"
    static_configs:
      - targets: ["<host>:<port>"]
```

Keep in mind that target defaults to `/metrics` endpoint. This will be relevant in a bit.

Start the docker container with Prometheus:

```bash
docker run -d \
  --name prometheus \
  -p 9090:9090 \
  -v $(pwd)/prometheus.yml:/etc/prometheus/prometheus.yml \
  prom/prometheus
```

Replace the `$(pwd)` with the proper path on Windows machine. Make sure to use absolute paths.

### Running Grafana

```bash
docker run -d \
  --name grafana \
  -p 3000:3000 \
  grafana/grafana
```

After Grafana is starter, it needs to be connected with Prometheus:
1. Open Grafana in a browser: http://localhost:3000.
1. Log in with default credentials (image defaults):
    - Username: admin
    - Password: admin
1. Add Prometheus as a data source:
    - Navigate to Connections → Data Sources → Add Data Source.
    - Choose Prometheus.
    - Set the URL to: http://host.docker.internal:9090 (on Windows/macOS) or http://<host_ip>:9090 (Linux). Port 9090 is taken from the previous step when launching Prometheus container.
1. Save and Test the connection.

### Creating custom metrics in dotnet

#### Tracking the error rate

Create custom counter:
```csharp
var meter = new Meter("ErrorMeter");
var errorCounter = meter.CreateCounter<long>("errors");
```

Add the meter:
```csharp
.WithMetrics(metrics =>
    {
        ...
        metrics.AddMeter("ErrorMeter");    
    });
```

Increase the counter via middleware:

```csharp
app.Use(
    (context, next) =>
    {
        try
        {
            return next();
        }
        catch
        {
            errorCounter.Add(1);
            throw;
        }
    }
);
```

You can see in the example above, that the metrics are glued together with the string name. So same mechanism can be used to add more custom metrics throughout your solution.

## Further reading

- https://sre.google/workbook/monitoring/